# Data Scraping from Wikipedia

Import the Required Libraries

In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


Install Beautifulsoup Library

In [8]:
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    certifi-2019.11.28         |           py36_0         156 KB  anaconda
    beautifulsoup4-4.8.2       |           py36_0         161 KB  anaconda
    soupsieve-1.9.5            |           py36_0          61 KB  anaconda
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.5-py36_0      an

Import the wikipedia page data in python object

In [56]:
import requests
import lxml
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL).text

Get the required table data in python object

In [59]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page,'html.parser')
My_table = soup.find('table',{'class':'wikitable sortable'})

Parse the table data in DataFrame

In [77]:
table_body = My_table.find('tbody')
data=[]
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])


In [86]:
column_names = ['PostalCode','Borough', 'Neighborhood'] 
dftable = pd.DataFrame(data,columns=column_names)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [187]:
new_df = dftable[dftable.Borough != 'Not assigned']
new_df['Neighborhood'][95]

'CFB Toronto'

In [ ]:
newdf1=new_df.drop([0],axis=0)

Reset the Index and drop unnecessary columns

In [125]:
newdf1.reset_index(inplace = True)

In [128]:
newdf1.head()

,level_0,index,PostalCode,Borough,Neighborhood
0,0,3,M3A,North York,Parkwoods
1,1,4,M4A,North York,Victoria Village
2,2,5,M5A,Downtown Toronto,Harbourfront
3,3,6,M6A,North York,Lawrence Heights
4,4,7,M6A,North York,Lawrence Manor


In [129]:
newdf1.drop("index", axis=1,inplace=True)

In [130]:
newdf1.drop("level_0", axis=1,inplace=True)

In [131]:
newdf1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [138]:
new2=newdf1.groupby(['PostalCode','Borough'])['Neighborhood'].apply(list)

In [142]:
newdf2=pd.DataFrame(new2)

In [170]:
newdf2['Neighborhood'] = [','.join(map(str, l)) for l in newdf2['Neighborhood']]

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [195]:
newdf2['Neighborhood'][93] = newdf2['Borough'][93]

In [201]:
newdf2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [202]:
newdf2.shape

(103, 3)